# Gaussian anomoly detection system for creditcard frauds single variate

In [112]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [113]:
data = pd.read_csv("creditcard.csv")
negData = data.groupby('Class').get_group(0)
posData = data.groupby('Class').get_group(1)
train, negCV, negTest = np.split(negData.sample(frac=1), [int(.6*len(negData)), int(.8*len(negData))])
posCV, posTest = np.split(posData.sample(frac=1), 2)
cv = negCV.append(posCV)
test = negTest.append(posTest)
Xtrain = train[train.columns[0:30]]
ytrain = train[train.columns[30:]]
XCV = cv[cv.columns[0:30]]
yCV = cv[cv.columns[30:]]
Xtest = test[test.columns[0:30]]
ytest = test[test.columns[30:]]
print("finished splitting data.......")

finished splitting data.......


In [115]:
mu, std = estimateGauss(Xtrain)
print("learned mu and sigma")

pCV = singlevariate(XCV,mu,std)
print("calculated p(x)")

epsilon , F1 = selectThreshold(pCV,yCV)

print ("Found best epsilon = " + str(epsilon) + ", best F1 = " + str(F1))

ptest = singlevariate(Xtest, mu, std) # Fit final model on test set

predictions = (ptest < epsilon).astype(int)
ytest = np.squeeze(ytest.values).astype(int)

tp = np.sum((predictions == 1).astype(int) & (ytest == 1).astype(int))
fp = np.sum((predictions == 1).astype(int) & (ytest == 0).astype(int))
fn = np.sum((predictions == 0).astype(int) & (ytest == 1).astype(int))
tn = np.sum((predictions == 0).astype(int) & (ytest == 0).astype(int))

prec = (tp * 1.0) / (tp + fp)
rec = (tp * 1.0) / (tp + fn)


print ("Precision = " + str(prec) + ", Recall = " + str(rec))

learned mu and sigma
calculated p(x)


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [114]:
def estimateGauss(X):
    mu = X.mean()
    m,n = X.shape
    std = np.zeros((1,n))
    for i in range(0,n):
                   std[0,i] += np.sum(np.square(X.iloc[:,i].values -mu[i]))
    std = std.reshape(n,)
    return mu, std
def singlevariate(X,mu,std):
    m ,n = X.shape
    p = np.zeros((m,n))
    a = np.ones((m,1))
    
    for i in range(0,n):
        r = std[i]*math.pow(2*math.pi,0.5)
        s = np.square(X.iloc[:,i].values-mu[i])
        
        t = 2*(std[i]**2)
        v = s/t
        u = np.exp(-v)
        p[:,i] = (1.0/r)*u
    #print(p)
    for i in range(0,m):
        for j in range(0,n):
            a[i,0] *= p[i,j]
    return a
def selectThreshold(yval, pval):
    yval = np.squeeze(yval.values).astype(int)

    bestEpsilon = 0.0
    bestF1 = 0.0

    stepsize = (max(pval) - min(pval)) / 1000
    for epsilon in np.arange(min(pval), max(pval), stepsize):
        predictions = (pval < epsilon).astype(int)

        tp = np.sum((predictions == 1).astype(int) & (yval == 1).astype(int))
        fp = np.sum((predictions == 1).astype(int) & (yval == 0).astype(int))
        fn = np.sum((predictions == 0).astype(int) & (yval == 1).astype(int))

        # calculate precision & recall
        prec = (tp * 1.0) / (tp + fp)
        rec = (tp * 1.0) / (tp + fn)

        F1 = (2 * prec * rec) * 1.0 / (prec + rec) # calculate F1 score using current epsilon

        if F1 > bestF1:
            bestF1 = F1
            bestEpsilon = epsilon

    return bestEpsilon, bestF1


In [111]:
mu , std = estimateGauss(Xtrain)
pCV = singlevariate(XCV,mu,std)
epsilon , F1 = selectThreshold(pCV,yCV)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [74]:
Xtrain.iloc[:,13].values

array([-0.71194594, -0.63810097, -0.33562009, ...,  0.36488969,
       -0.3518013 ,  0.81192358])

In [69]:
np.mean(Xtrain.values)

3167.5221457862294